In [1]:
%run PATTERN_OBSERVER.ipynb

In [2]:
from ipywidgets import Layout,HBox,VBox,Checkbox,Button
from ipywidgets import Color # pretty color for apply button
from typing import List,Dict

In [53]:
'''
WHAT
- class to generate a list of checkboxes
'''
class CHECKBOXES(ISubject,IObserver):
    ####################################
    # constructor
    ####################################
    def __init__(self,
        name='CHECKBOX LIST',
        options={'option '+str(x):str(x)*10 for x in range(10)},  # Dict[str,object] used to make option_widget_list, always kept up-to-date
        default=False,
        width='auto',
        ):
        self._default=default
        self._width=width
        self._options=options
        
        self.name=name
        self.widget=self._make()
        
        self.applied_settings=self._current_settings # update reference
        self._observers=set() # for observer pattern
        
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None:
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.add(observer)
        
    def detach(self,observer : IObserver) -> None:
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self) -> None:
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,self.applied_settings)
        
    # observer
    def react(self,subject_name,subject_info):
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)
        
        if subject_name=='SETTINGS':
            self._refresh_options(subject_info['all_backtests'])
        
    ####################################
    # buttons
    ####################################
    def _button_inactive(self,button):
        button.style.button_color=Color(None).name
        button.disabled=True
        
    def _button_active(self,button,color='pink'):
        button.style.button_color=color
        button.disabled=False
        
    def _update_button_state(self):
        if self.applied_settings==self._current_settings:
            # if nothing changed
            self._button_inactive(self._control_widgets[0])
            self._button_inactive(self._control_widgets[1])
        else:
            # if something changed
            self._button_active(self._control_widgets[0],color='lightgreen')
            self._button_active(self._control_widgets[1],color='pink')

    '''NB - need apply button to trigger a change, if use checkbox change to trigger plotting then clicking "all/none/invert" causes multiple sequential intra plots'''
    def _click_apply(self,change):
        print(self.name,':','CLICKED',change.description)

        self.applied_settings=self._current_settings # update checkpoint
        self._update_button_state() # update button state
        self._logging() # show internals
        self.notify() # notify observers
        
    def _logging(self):
        for k,v in self.applied_settings.items():
            if v:
                print(self.name,':','REFERENCE',':',k,'=','True')

    def _click_cancel(self,change):
        print(self.name,':','CLICKED',change.description)

        # reset value
        for w,(k,value) in zip(self._option_widgets,self.applied_settings.items()):
            w.value=value

        self._update_button_state() # update button state

    def _click_none(self,change):
        print(self.name,':','CLICKED',change.description)
        
        for c in self._option_widgets:
            c.value=False
            
    def _click_all(self,change):
        print(self.name,':','CLICKED',change.description)
        
        for c in self._option_widgets:
            c.value=True
            
    def _click_invert(self,change):
        print(self.name,':','CLICKED',change.description)
        
        for c in self._option_widgets:
            c.value=not c.value

    def _onchange_checkbox(self,change):
        print(self.name,':','CHANGED',change.owner.description,'=',change.new)
        self._update_button_state() # update button state
    
    def _make(self):
        # pretty controls
        controls=HBox(self._make_controls())
        
        # pretty checkboxes
        options=VBox(
            children=self._make_options(self._options),
            layout=Layout(height='300px')
        )
        
        # return
        return VBox(
            children=[controls,options],
            layout=Layout(border='1px solid black',width=self._width,overflow_x='scroll')
        )
    
    def _make_controls(self):
        # checkbox controls
        button_apply=Button(description='apply',disabled=True)
        button_apply.on_click(self._click_apply)
        
        button_cancel=Button(description='cancel',disabled=True)
        button_cancel.on_click(self._click_cancel)
        
        button_all=Button(description='all')
        button_all.on_click(self._click_all)
        
        button_none=Button(description='none')
        button_none.on_click(self._click_none)
        
        button_invert=Button(description='invert')
        button_invert.on_click(self._click_invert)

        return [
            button_apply,
            button_cancel,
            button_all,
            button_none,
            button_invert,
        ]

    def _make_options(self,options : Dict[str,object]):
        # make list of checkboxes
        widgets=[
            Checkbox(
                description=option,
                layout=Layout(height='15px'),
                indent=False,
                value=self._default
            ) for option in options
        ]
        
        # register checkboxes
        for c in widgets:
            c.observe(self._onchange_checkbox,names='value')

        # return
        return widgets
    
    @property
    def _control_widgets(self):
        return self.widget.children[0].children
    
    @property
    def _option_widgets(self):
        return self.widget.children[1].children
    
    @_option_widgets.setter
    def _option_widgets(self,new_options : List[Checkbox]):
        self.widget.children[1].children=new_options
        
    @property
    def _current_settings(self):
        return {(k,v):w.value for w,(k,v) in zip(self._option_widgets,self._options.items())}
    
    def _refresh_options(self,options):
        print(self.name,':','refreshing options')
        
        self._option_widgets=self._make_options(options) # update existing children widgets
        self.applied_settings=self._current_settings # set again from fresh
        self._update_button_state() # update button state
        self.notify() # notify observers

# example
CHECKBOXES().widget

In [58]:
'''
a=CHECKBOXES()
a.widget

a.applied_settings

a._current_settings
'''
None

In [32]:
'''
# test setter
a=CHECKBOXES()
a.widget
a._refresh_options(list('abcdef'))
'''
None